## Section 4.4: Examining High-Level Compositinality

In [1]:
from __future__ import print_function

from torchtext import data, datasets
import torch
import os
from collections import Counter
import torch.nn.functional as F
import torch.autograd as autograd
import torch.nn as nn

In [2]:
# preserves case of words
inputs = data.Field(lower='preserve-case')

# No tokenization applied because the data is not seq
# unk_token=None: ignore out of vocabulary tokens, since these are grades
answers = data.Field(sequential=False, unk_token=None) # y: floats

# fine_grained=False - use the following grade mapping { 0,1 -> negativ; 2 -> neutral; 3,4 -> positive }
# filter=... - remove the neutral class to reduce the problem to binary classification
# train_subtrees=False - Use only complete review instead of also using subsentences (subtrees)
train, dev, test = datasets.SST.splits(inputs, answers, fine_grained = False, train_subtrees = True,
                                       filter_pred=lambda ex: ex.label != 'neutral')
# build the initial vocabulary from the SST dataset
inputs.build_vocab(train, dev, test)

# then enhance it with the pre-trained glove model 
inputs.vocab.load_vectors('glove.6B.300d')

# build the vocab for the labels (only consists of 'positive','negative')
answers.build_vocab(train)

# You can use these iterators to train/test/validate the network :)
train_iter, dev_iter, test_iter = data.BucketIterator.splits(
        (train, dev, test), repeat=False, batch_size=1, device=-1)

/home/ckerriou/anaconda2/lib/python2.7/site-packages/nltk/tree.py:623: UnicodeWarning: Unicode equal comparison failed to convert both arguments to Unicode - interpreting them as being unequal
  elif token == close_b:
/home/ckerriou/anaconda2/lib/python2.7/site-packages/nltk/tree.py:616: UnicodeWarning: Unicode equal comparison failed to convert both arguments to Unicode - interpreting them as being unequal
  if token[0] == open_b:


In [7]:
for _,batch in enumerate(train_iter):
    b = batch
    break

In [8]:
model =  torch.load('model.pt', map_location=lambda storage, loc: storage)

In [9]:
word_vecs = model.word_embeddings(batch.text)[:,0].data

In [10]:
word_vecs


-2.6928e-01  2.2387e-01  1.0296e-01  ...  -1.9804e-01 -1.3537e-01 -3.3289e-02
-2.8300e-01  2.2680e-01  4.2434e-01  ...  -2.9751e-01 -5.7114e-01 -1.1897e-02
-1.2959e-01 -4.5655e-01 -1.6903e-01  ...  -2.1640e-02  1.7986e-01 -4.5405e-01
                ...                   ⋱                   ...                
-2.7511e-01 -1.0711e-01 -3.7974e-01  ...  -3.8723e-01  3.1628e-01  3.1824e-01
-2.8478e-01  2.9991e-02  4.7464e-01  ...  -1.7845e-01  5.2008e-01 -7.2839e-01
-8.7050e-02 -4.5883e-02  2.0488e-01  ...  -1.6052e-01  2.2561e-02  1.1160e-01
[torch.FloatTensor of size 11x300]

In [11]:
model.word_embeddings(batch.text)

Variable containing:
( 0 ,.,.) = 
 -2.6928e-01  2.2387e-01  1.0296e-01  ...  -1.9804e-01 -1.3537e-01 -3.3289e-02

( 1 ,.,.) = 
 -2.8300e-01  2.2680e-01  4.2434e-01  ...  -2.9751e-01 -5.7114e-01 -1.1897e-02

( 2 ,.,.) = 
 -1.2959e-01 -4.5655e-01 -1.6903e-01  ...  -2.1640e-02  1.7986e-01 -4.5405e-01
... 

( 8 ,.,.) = 
 -2.7511e-01 -1.0711e-01 -3.7974e-01  ...  -3.8723e-01  3.1628e-01  3.1824e-01

( 9 ,.,.) = 
 -2.8478e-01  2.9991e-02  4.7464e-01  ...  -1.7845e-01  5.2008e-01 -7.2839e-01

(10 ,.,.) = 
 -8.7050e-02 -4.5883e-02  2.0488e-01  ...  -1.6052e-01  2.2561e-02  1.1160e-01
[torch.FloatTensor of size 11x1x300]